In [3]:
# Import Libraries

import os
import requests
import psycopg2
import json
import arcpy

# Get CWD

cwd = os.getcwd() # This is a global variable for where the notebook is (must change if running in arcpro)

# Create GeoDataBase
# This is the communal GeoDataBase, only run once

if not os.path.exists(os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')): # If it doesn't exist, create it

    arcpy.management.CreateFileGDB(os.path.join(cwd, '..', '..', 'data'), 'QAQC')

# Make it workspace

arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

arcpy.env.overwriteOutput = True # Overwrite layers is okay

In [4]:
# Get Data

# Make a GET request to the URL and parse the response as JSON
url = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Planning_Primary_Zoning/FeatureServer/0/query?outFields=ZONE_CODE&where=1%3D1&f=geojson'
response = requests.get(url)
data = json.loads(response.content)

# Find overlapping polygons (Needs work)

In [5]:
# set the input URL. Different then one from above
input_url = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Planning_Primary_Zoning/FeatureServer/0'

# create a feature layer from the input URL
arcpy.MakeFeatureLayer_management(input_url, "input_layer8")

# find overlapping polygons with the intersect tool
output_layer8 = arcpy.Intersect_analysis(["input_layer8", "input_layer8"], "memory\\output_layer8")

# print the number of features in the output layer
print(arcpy.GetCount_management(output_layer8))

# create a dictionary to store the overlapping features
overlapping_features = {}

# loop through the output layer and populate the dictionary
with arcpy.da.SearchCursor(output_layer8, ['OID@', 'SHAPE@']) as cursor:
    for row in cursor:
        oid, shape = row
        if oid not in overlapping_features:
            overlapping_features[oid] = [shape]
        else:
            overlapping_features[oid].append(shape)

# loop through the overlapping features and print their object IDs
for oid, shapes in overlapping_features.items():
    if len(shapes) > 1:
        print(oid)

# delete the output layer from memory
arcpy.Delete_management(output_layer8)

2215


<Result 'true'>

# Find null values in data and put errors to an error code column

In [34]:
features = data['features']

for feature in features:
    zone_code = feature['properties']['ZONE_CODE']
    feature['properties']['ERROR_CODE'] = 'None' # Adding an error code property
    geometry = feature['geometry']
    
    # Check for null values in zone_code and geometry
    if zone_code is None:
        feature['properties']['ERROR_CODE'] = 'ZoneError'
    if len(str(geometry['coordinates'])) < 6: # This would be empty coordinates
        errorfeature['properties']['ERROR_CODE'] = 'GeomError'


# Upload to database and local GeoDataBase

In [45]:
### Locally

table_name = 'mpls_zoning'

# Save as geojson

savepath = os.path.join(cwd, '..', '..', 'data', 'zoning.geojson')

with open(savepath, 'w') as f:
    f.write(json.dumps(data))
    
zoning_temp = arcpy.conversion.JSONToFeatures(savepath, table_name + '_wgs')

# Reproject

out_coordinate_system = arcpy.SpatialReference(26915)
zoning = arcpy.Project_management(table_name + '_wgs', table_name, out_coordinate_system)

In [ ]:
### Database

# Get database credentials

cred_pth = os.path.join(os.getcwd(), '..', '..', 'database', 'db_credentials.txt')

with open(cred_pth, 'r') as f:
    
    creds = f.readlines()[0].rstrip('\n').split(', ')
    
# Connect to PostGIS Database

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

conn = psycopg2.connect(**pg_connection_dict)

cur = conn.cursor()

# Insert into Database

for feature in features:
    zone_code = feature['properties']['ZONE_CODE']
    error_code = feature['properties']['ERROR_CODE']
    geometry = json.dumps(feature['geometry'])
    
    # Set the SRID of the geometry to 4326
    geometry = "ST_SetSRID(ST_GeomFromGeoJSON('{}'), 4326)".format(geometry)
    
    cur.execute("INSERT INTO MPLSZoning (zone_code, error_code, geometry) VALUES (%s, %s, {})".format(geometry), (zone_code, error_code))
    conn.commit()

cur.close()
conn.close()


: 

: 